In [2]:
import os
import shutil

def find_and_copy_missing_images(valid_folder, label_folder, destination_folder):
    """
    Ищет изображения в папке label, которых нет в папке valid (по имени файла без расширения),
    и копирует их в новую папку.

    Args:
        valid_folder (str): Путь к папке с "валидными" файлами.
        label_folder (str): Путь к папке с изображениями, которые нужно проверить.
        destination_folder (str): Путь к папке, куда будут скопированы недостающие изображения.
    """
    # Создаем целевую папку, если она не существует
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
        print(f"Создана папка: {destination_folder}")

    # Получаем список базовых имен файлов (без расширения) из папки valid
    valid_basenames = set()
    if os.path.exists(valid_folder):
        for filename in os.listdir(valid_folder):
            if os.path.isfile(os.path.join(valid_folder, filename)):
                name, _ = os.path.splitext(filename)
                valid_basenames.add(name)
        print(f"Найдено {len(valid_basenames)} уникальных базовых имен в папке {valid_folder}")
    else:
        print(f"Папка {valid_folder} не найдена.")
        return

    # Проходим по файлам в папке label и ищем те, которых нет в valid
    copied_count = 0
    if os.path.exists(label_folder):
        for filename in os.listdir(label_folder):
            label_filepath = os.path.join(label_folder, filename)
            if os.path.isfile(label_filepath):
                name, ext = os.path.splitext(filename)

                # Проверяем, является ли файл изображением (можно добавить больше расширений)
                if ext.lower() in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif']:
                    # Если базового имени файла из label нет в списке базовых имен из valid
                    if name not in valid_basenames:
                        destination_filepath = os.path.join(destination_folder, filename)
                        try:
                            shutil.copy2(label_filepath, destination_filepath) # copy2 сохраняет метаданные
                            print(f"Скопировано: {filename}")
                            copied_count += 1
                        except Exception as e:
                            print(f"Ошибка при копировании файла {filename}: {e}")
                # else:
                #     print(f"Пропущен файл {filename} (не похоже на изображение)")

        print(f"\nЗавершено. Скопировано {copied_count} изображений в папку {destination_folder}.")

    else:
        print(f"Папка {label_folder} не найдена.")

# --- Использование скрипта ---
# Укажите пути к вашим папкам
valid_directory = 'C:/Users/user/Desktop/valid'
label_directory = 'C:/Users/user/Desktop/Work/DeepLearning/JCB/images_dataset/label'
new_destination_directory = 'C:/Users/user/Desktop/for_valid'

# Запускаем функцию
find_and_copy_missing_images(valid_directory, label_directory, new_destination_directory)

Найдено 316 уникальных базовых имен в папке C:/Users/user/Desktop/valid
Скопировано: 058242.tiff
Скопировано: 075100.tiff
Скопировано: 075115.tiff
Скопировано: 078281-2.tiff
Скопировано: 078282.tiff
Скопировано: 081681.tiff
Скопировано: 09-20520.tiff
Скопировано: 094080.tiff
Скопировано: 103401.tiff
Скопировано: 11-10210.tiff
Скопировано: 11-10211.tiff
Скопировано: 12-13740.tiff
Скопировано: 121481.tiff
Скопировано: 121483.tiff
Скопировано: 125181.tiff
Скопировано: 126-00187-4.tiff
Скопировано: 126-00187-5.tiff
Скопировано: 126-00232-3.tiff
Скопировано: 126-00232-4.tiff
Скопировано: 126-00232-6.tiff
Скопировано: 126-01484-9.tiff
Скопировано: 126-01545-9.tiff
Скопировано: 126-25600-9.tiff
Скопировано: 126-25600-9A.tiff
Скопировано: 128-10470-4.tiff
Скопировано: 128-10850.tiff
Скопировано: 128-11309-7.tiff
Скопировано: 128-15483-4.tiff
Скопировано: 128-16535-6.tiff
Скопировано: 128-C0010-1.tiff
Скопировано: 128-C0011-5.tiff
Скопировано: 128-C9958-1.tiff
Скопировано: 128-D0043-2.tiff
Скоп

In [5]:
import pandas as pd

def filter_files_by_csv(folder_path, csv_filepath, name_column_index=0):
    """
    Оставляет в папке только те файлы, базовые имена которых (без расширения)
    перечислены в указанном CSV файле. Остальные файлы удаляются.

    Args:
        folder_path (str): Путь к папке, содержащей файлы для фильтрации.
        csv_filepath (str): Путь к CSV файлу со списком имен файлов, которые нужно оставить.
        name_column_index (int): Индекс колонки в CSV (начиная с 0), где находятся
                                 имена файлов (без расширения). По умолчанию 0 (первая колонка).
    """
    # Проверяем существование папки
    if not os.path.isdir(folder_path):
        print(f"Ошибка: Папка '{folder_path}' не найдена.")
        return

    df = pd.read_csv(csv_filepath)

    # Читаем разрешенные имена файлов из CSV
    df = df["drawing_name"].unique().tolist()
    allowed_basenames = set(df)

    deleted_count = 0
    total_files = 0
    try:
        for filename in os.listdir(folder_path):
            filepath = os.path.join(folder_path, filename)
            if os.path.isfile(filepath):
                total_files += 1
                name, _ = os.path.splitext(filename) # Получаем базовое имя файла
                # Проверяем, есть ли базовое имя файла в списке разрешенных
                if name not in allowed_basenames:
                    try:
                        os.remove(filepath)
                        print(f"Удален файл: {filename}")
                        deleted_count += 1
                    except Exception as e:
                        print(f"Ошибка при удалении файла {filename}: {e}")

        print(f"\nЗавершено. Всего файлов проверено: {total_files}.")
        print(f"Удалено файлов: {deleted_count}.")
        print(f"Осталось файлов: {total_files - deleted_count}.")

    except Exception as e:
        print(f"Ошибка при обработке файлов в папке: {e}")


# # --- Использование скрипта ---
# # Укажите путь к папке, которую нужно очистить
target_folder = 'C:/Users/user/Desktop/for_valid'

# # Укажите путь к вашему CSV файлу
csv_file = 'C:/Users/user/Desktop/Work/DeepLearning/JCB/merged_result.csv'

# # Укажите индекс колонки в CSV, где находятся имена файлов (начиная с 0)
# # Если имена в первой колонке, оставьте 0
name_column = 0

# # Запускаем функцию фильтрации
filter_files_by_csv(target_folder, csv_file, name_column)



Удален файл: 075115.tiff
Удален файл: 09-20520.tiff
Удален файл: 11-10210.tiff
Удален файл: 11-10211.tiff
Удален файл: 12-13740.tiff
Удален файл: 126-00187-4.tiff
Удален файл: 126-00187-5.tiff
Удален файл: 126-00232-3.tiff
Удален файл: 126-00232-4.tiff
Удален файл: 126-00232-6.tiff
Удален файл: 126-01484-9.tiff
Удален файл: 126-01545-9.tiff
Удален файл: 126-25600-9.tiff
Удален файл: 126-25600-9A.tiff
Удален файл: 128-10470-4.tiff
Удален файл: 128-10850.tiff
Удален файл: 128-11309-7.tiff
Удален файл: 128-15483-4.tiff
Удален файл: 128-16535-6.tiff
Удален файл: 128-C0010-1.tiff
Удален файл: 128-C0011-5.tiff
Удален файл: 128-C9958-1.tiff
Удален файл: 128-E0718-1a.tiff
Удален файл: 128-E3118-2.tiff
Удален файл: 128-E3119-1.tiff
Удален файл: 128-E8557-2A.tiff
Удален файл: 128-E8899-1.tiff
Удален файл: 128-E9000-1.tiff
Удален файл: 128-F4148-3.tiff
Удален файл: 128-F4149-3.tiff
Удален файл: 128-G2641-1.tiff
Удален файл: 128-H3275-4.tiff
Удален файл: 128-L0767-1.tiff
Удален файл: 128-L4837-2.t